In [ ]:
# qlogin -q short.qg -l gpu=1[affinity=true],gputype=rtx8000
# # qlogin -q short.qg -l gpu=1[affinity=true],gputype=a100

# srun -p gpu_short --gres gpu:1 --constraint='a100-pcie-80gb|quadro-rtx8000|a100-pcie-40gb' --pty bash
srun -p gpu_short --gres gpu:1 --constraint='a100-pcie-80gb' --pty bash

In [ ]:
#!/bin/bash

cd /well/win/users/hsv459/agemapper

module purge
module load Python/3.7.4-GCCcore-8.3.0

source /well/win-fmrib-analysis/users/hsv459/python/swinagemapper-skylake/bin/activate
echo "Loaded swinagemapper-skylake environment."

# continue to use your python venv as normal
   
ipython



In [ ]:
import os
import shutil
import argparse
import logging

import torch
import torch.nn as nn
import torch.utils.data as data
import numpy as np

from solver import Solver
from utils.data_utils import get_datasets_dynamically, get_test_datasets_dynamically
from utils.settings import Settings
import utils.data_evaluation as evaluations
from utils.misc import create_folder

from AgeMapper import AgeMapper

import pandas as pd
from collections import OrderedDict

MSELoss = nn.MSELoss()

In [ ]:

networks=[
    'AC3-1',
    'AC2-6',
    'AC3-3',
    'AC1-5',
    'AC5-7',
    'AC6-8',
    'AC6-16',
]


noiseSTD = [
    0,0.01,0.02,0.04,0.06,0.08,
    0.1,0.12,0.14,0.16,0.18,0.2
]

def load_data_dynamically(data_parameters, mapping_evaluation_parameters=None, flag='train'):
    
    if flag=='train':
        print("Data is loading...")
        train_data, validation_data, resolution = get_datasets_dynamically(data_parameters)
        print("Data has loaded!")
        print("Training dataset size is {}".format(len(train_data)))
        print("Validation dataset size is {}".format(len(validation_data)))
        return train_data, validation_data, resolution
    elif flag=='test':
        print("Data is loading...")
        test_data, volumes_to_be_used, prediction_output_statistics_name, resolution = get_test_datasets_dynamically(data_parameters, mapping_evaluation_parameters)
        print("Data has loaded!")
        len_test_data = len(test_data)
        print("Testing dataset size is {}".format(len_test_data))
        return test_data, volumes_to_be_used, prediction_output_statistics_name, len_test_data, resolution
    else:
        print('ERROR: Invalid Flag')
        return None
    
def _load_mean(dataset_sex):

    if dataset_sex == 'male':
        mean_age = 64.64810970818492
    else:
        mean_age = 63.370316719492536

    mean_age = np.array(np.float32(mean_age))

    return mean_age 

def _statistics_calculator(output_age, target_age):

    output_age = np.array(output_age)
    target_age = np.array(target_age)
    age_delta = output_age - target_age
    loss = MSELoss(torch.from_numpy(output_age), torch.from_numpy(target_age)).numpy()

    return age_delta, loss


for idx in range(len(networks)):
    
    
    # Find the pretrained network weights
    
    network = networks[idx]
    
    print('NETWORK: {}'.format(network))

    network_path = '/well/win-fmrib-analysis/users/hsv459/agemapper/saved_models/'
    network_name = network + '.pth.tar'
    network_path += network_name
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    trained_model = torch.load(network_path, map_location=torch.device(device))

    
    # Load the settings
    
    settings_file_name = network + '.ini'
    evaluation_settings_file_name = network + '_eval.ini'
    settings = Settings(settings_file_name)
    settings_evaluation = Settings(evaluation_settings_file_name)

    mapping_evaluation_parameters = settings_evaluation['MAPPING']
    data_parameters = settings['DATA']
    training_parameters = settings['TRAINING']
    network_parameters = settings['NETWORK']
    misc_parameters = settings['MISC']
    
    # Get the pretrained model resolution
    
    data_directory = data_parameters['data_directory']
    modality_flag = data_parameters['modality_flag']
    scaling_values_simple = pd.read_csv(data_parameters['scaling_values'], index_col=0)
    resolution = scaling_values_simple.loc[modality_flag].resolution
    del scaling_values_simple

    # Get the pretrained model ready

    dropout_rate_1 = network_parameters['dropout_rate_1']
    dropout_rate_2 = network_parameters['dropout_rate_2']
    dropout_rate_3 = network_parameters['dropout_rate_3']

    model = AgeMapper(resolution=resolution,
                                    dropout_rate_1=dropout_rate_1,
                                    dropout_rate_2=dropout_rate_2,
                                    dropout_rate_3=dropout_rate_3,
                                    )

    model.load_state_dict(trained_model)

    del trained_model, network_path, network_name

    if torch.cuda.is_available() == True and device!='cpu':
        model.cuda(device)

    model.eval();
    
    
    # Prepare the output folder and several other settings
    
    mapping_evaluation_parameters['experiment_name'] = mapping_evaluation_parameters['experiment_name'] + '_noise'
    experiment_name = mapping_evaluation_parameters['experiment_name']
    prediction_output_path = experiment_name + "_predictions"
    create_folder(prediction_output_path)
    
    
    trained_model_path = "saved_models/" + experiment_name + ".pth.tar"
    control = mapping_evaluation_parameters['control']
    dataset_sex = data_parameters['dataset_sex']
    
    data_parameters['noise_mean'] = 0.
    
    
    # Start a loop going through the various noise settings
    
    for std in noiseSTD:
        
        print('----> std=',std)
        
        data_parameters['noise_std'] = std
        
        test_data, volumes_to_be_used, prediction_output_statistics_name, len_test_data, resolution = load_data_dynamically(
                                                data_parameters=data_parameters, 
                                                mapping_evaluation_parameters=mapping_evaluation_parameters, 
                                                flag='test'
                                                )
        
        test_loader = data.DataLoader(
            dataset = test_data,
            batch_size=1,
            shuffle=False,
            pin_memory=True,
            num_workers=data_parameters['num_workers']
        )

        output_statistics = {}
        output_statistics_path = os.path.join(prediction_output_path, prediction_output_statistics_name + '_' + str(std))

        with torch.no_grad():

            for batch_index, sampled_batch in enumerate(test_loader):
                X = sampled_batch[0].type(torch.FloatTensor)
                y_age = sampled_batch[1].type(torch.FloatTensor)
                y_age = (y_age.cpu().numpy()).astype('float32')
                y_age = np.squeeze(y_age)
                subject = volumes_to_be_used[batch_index]

                # We add an extra dimension (~ number of channels) for the 3D convolutions.
                if len(X.size())<5:
                    X = torch.unsqueeze(X, dim=1)

                if torch.cuda.is_available():
                    X = X.cuda(device, non_blocking=True)

                y_hat = model(X)   # Forward pass
                y_hat = (y_hat.cpu().numpy()).astype('float32')
                y_hat = np.squeeze(y_hat)

                target_age = y_age

                age_delta, loss = _statistics_calculator(y_hat, target_age)
                output_statistics[subject] = [target_age, y_hat, age_delta, loss]

                print("\r Processed {:.3f}%: {}/{} subjects".format((batch_index+1)/len_test_data * 100.0, batch_index+1, len_test_data), end='')

            output_statistics_df = pd.DataFrame.from_dict(output_statistics, orient='index', columns=['target_age', 'output_age', 'age_delta', 'loss'])     
            output_statistics_df.to_csv(output_statistics_path)

        print('\n')


In [ ]:
def load_data_dynamically(data_parameters, mapping_evaluation_parameters=None, flag='train'):
    
    if flag=='train':
        print("Data is loading...")
        train_data, validation_data, resolution = get_datasets_dynamically(data_parameters)
        print("Data has loaded!")
        print("Training dataset size is {}".format(len(train_data)))
        print("Validation dataset size is {}".format(len(validation_data)))
        return train_data, validation_data, resolution
    elif flag=='test':
        print("Data is loading...")
        test_data, volumes_to_be_used, prediction_output_statistics_name, resolution = get_test_datasets_dynamically(data_parameters, mapping_evaluation_parameters)
        print("Data has loaded!")
        len_test_data = len(test_data)
        print("Testing dataset size is {}".format(len_test_data))
        return test_data, volumes_to_be_used, prediction_output_statistics_name, len_test_data, resolution
    else:
        print('ERROR: Invalid Flag')
        return None
    
def _load_mean(dataset_sex):

    if dataset_sex == 'male':
        mean_age = 64.64810970818492
    else:
        mean_age = 63.370316719492536

    mean_age = np.array(np.float32(mean_age))

    return mean_age 

def _statistics_calculator(output_age, target_age):

    output_age = np.array(output_age)
    target_age = np.array(target_age)
    age_delta = output_age - target_age
    loss = MSELoss(torch.from_numpy(output_age), torch.from_numpy(target_age)).numpy()

    return age_delta, loss


In [ ]:

for idx in range(len(networks)):
    
    
    # Find the pretrained network weights
    
    network = networks[idx]
    
    print('NETWORK: {}'.format(network))

    network_path = '/well/win-fmrib-analysis/users/hsv459/agemapper/saved_models/'
    network_name = network + '.pth.tar'
    network_path += network_name
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    trained_model = torch.load(network_path, map_location=torch.device(device))

    
    # Load the settings
    
    settings_file_name = network + '.ini'
    evaluation_settings_file_name = network + '_eval.ini'
    settings = Settings(settings_file_name)
    settings_evaluation = Settings(evaluation_settings_file_name)

    mapping_evaluation_parameters = settings_evaluation['MAPPING']
    data_parameters = settings['DATA']
    training_parameters = settings['TRAINING']
    network_parameters = settings['NETWORK']
    misc_parameters = settings['MISC']
    
    # Get the pretrained model resolution
    
    data_directory = data_parameters['data_directory']
    modality_flag = data_parameters['modality_flag']
    scaling_values_simple = pd.read_csv(data_parameters['scaling_values'], index_col=0)
    resolution = scaling_values_simple.loc[modality_flag].resolution
    del scaling_values_simple

    # Get the pretrained model ready

    dropout_rate_1 = network_parameters['dropout_rate_1']
    dropout_rate_2 = network_parameters['dropout_rate_2']
    dropout_rate_3 = network_parameters['dropout_rate_3']

    model = AgeMapper(resolution=resolution,
                                    dropout_rate_1=dropout_rate_1,
                                    dropout_rate_2=dropout_rate_2,
                                    dropout_rate_3=dropout_rate_3,
                                    )

    model.load_state_dict(trained_model)

    del trained_model, network_path, network_name

    if torch.cuda.is_available() == True and device!='cpu':
        model.cuda(device)

    model.eval();
    
    
    # Prepare the output folder and several other settings
    
    mapping_evaluation_parameters['experiment_name'] = mapping_evaluation_parameters['experiment_name'] + '_noise'
    experiment_name = mapping_evaluation_parameters['experiment_name']
    prediction_output_path = experiment_name + "_predictions"
    create_folder(prediction_output_path)
    
    
    trained_model_path = "saved_models/" + experiment_name + ".pth.tar"
    control = mapping_evaluation_parameters['control']
    dataset_sex = data_parameters['dataset_sex']
    
    data_parameters['noise_mean'] = 0.
    
    
    # Start a loop going through the various noise settings
    
    for std in noiseSTD:
        
        print('----> std=',std)
        
        data_parameters['noise_std'] = std
        
        test_data, volumes_to_be_used, prediction_output_statistics_name, len_test_data, resolution = load_data_dynamically(
                                                data_parameters=data_parameters, 
                                                mapping_evaluation_parameters=mapping_evaluation_parameters, 
                                                flag='test'
                                                )
        
        test_loader = data.DataLoader(
            dataset = test_data,
            batch_size=1,
            shuffle=False,
            pin_memory=True,
            num_workers=data_parameters['num_workers']
        )

        output_statistics = {}
        output_statistics_path = os.path.join(prediction_output_path, prediction_output_statistics_name + '_' + str(std))

        with torch.no_grad():

            for batch_index, sampled_batch in enumerate(test_loader):
                X = sampled_batch[0].type(torch.FloatTensor)
                y_age = sampled_batch[1].type(torch.FloatTensor)
                y_age = (y_age.cpu().numpy()).astype('float32')
                y_age = np.squeeze(y_age)
                subject = volumes_to_be_used[batch_index]

                # We add an extra dimension (~ number of channels) for the 3D convolutions.
                if len(X.size())<5:
                    X = torch.unsqueeze(X, dim=1)

                if torch.cuda.is_available():
                    X = X.cuda(device, non_blocking=True)

                y_hat = model(X)   # Forward pass
                y_hat = (y_hat.cpu().numpy()).astype('float32')
                y_hat = np.squeeze(y_hat)

                target_age = y_age

                age_delta, loss = _statistics_calculator(y_hat, target_age)
                output_statistics[subject] = [target_age, y_hat, age_delta, loss]

                print("\r Processed {:.3f}%: {}/{} subjects".format((batch_index+1)/len_test_data * 100.0, batch_index+1, len_test_data), end='')

            output_statistics_df = pd.DataFrame.from_dict(output_statistics, orient='index', columns=['target_age', 'output_age', 'age_delta', 'loss'])     
            output_statistics_df.to_csv(output_statistics_path)

        print('\n')
